# TP1 - Finding Keys using Functional Dependencies 
--------------------------

In this lab, we are going to learn 

- how to use Jupyter
- how to use SQLite
- how to discover keys in relations

## How to use Jupyter

To execute a cell, click on it, and then use SHIFT+ENTER (this will execute the contents of the cell, and move down to the next one!)

To edit a cell, click on it. If the cell uses markdown code, then use ENTER to edit it.

See the Help menu for other useful keyboard commands. You can always use the menu bar instead as well.


In [1]:
print("Hello world!")

Hello world!


Another example:

In [2]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


#### Exercise 1

Print numbers 1 to 9 in reverse order

In [3]:
for i in reversed(range(1, 10)):
    print(i)

9
8
7
6
5
4
3
2
1


## How to use SQLite

To work with SQL easily in a notebook, we'll load the ipython-sql extension as follows:

In [4]:
%load_ext sql
%sql sqlite://

'Connected: @None'

We are going to create a table:

In [5]:
%%sql DROP TABLE IF EXISTS T;
CREATE TABLE T(course VARCHAR, classroom INT, time INT);
INSERT INTO T VALUES ('CS 364', 132, 900);
INSERT INTO T VALUES ('CS 245', 140, 1000);
INSERT INTO T VALUES ('EE 101', 210, 900);

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Let's run our first SQL query:

In [6]:
%sql SELECT * FROM T;

 * sqlite://
Done.


course,classroom,time
CS 364,132,900
CS 245,140,1000
EE 101,210,900


#### Exercise 2

List the name of the courses with time less than 950

In [7]:
%sql SELECT course FROM T WHERE time < 950

 * sqlite://
Done.


course
CS 364
EE 101


## How to discover keys in relations

Now, we are going to work with functional dependencies, keys and closures. Our final goal is going to build a method to find keys in a relation.

### Functional Dependencies

Recall that given a set of attributes  $\{A_1, \dots, A_n\}$ and a set of FDs $\Gamma$

The closure, denoted $\{A_1, \dots, A_n\}^+$, is defined to be the largest set of attributes B s.t. $$A_1,\dots,A_n \rightarrow B \text{ using } \Gamma.$$

We're going to use some functions to compute the closure of a set of attributes and other such operations (_from CS145 Stanford_)

In [8]:
# Source code


def to_set(x):
    """Convert input int, string, list, tuple, set -> set"""
    if type(x) == set:
        return x
    elif type(x) in [list, set]:
        return set(x)
    elif type(x) in [str, int]:
        return set([x])
    else:
        raise Exception("Unrecognized type.")


def fd_to_str(lr_tuple):
    lhs = lr_tuple[0]
    rhs = lr_tuple[1]
    return ",".join(to_set(lhs)) + " -> " + ",".join(to_set(rhs))


def fds_to_str(fds):
    return "\n\t".join(map(fd_to_str, fds))


def set_to_str(x):
    return "{" + ",".join(x) + "}"


def fd_applies_to(fd, x):
    lhs, rhs = map(to_set, fd)
    return lhs.issubset(x)


def print_setup(A, fds):
    print("Attributes = " + set_to_str(A))
    print("FDs = \t" + fds_to_str(fds))


def print_fds(fds):
    print("FDs = \t" + fds_to_str(fds))

Now, let's look at a concrete example. For example, the code for

attributes = { name, category, color, department, price}

and functional dependencies:

name $\rightarrow$ color

category $\rightarrow$ department

color, category $\rightarrow$ price

is the following:

In [9]:
# These are the attribute set.
attributes = set(["name", "category", "color", "department", "price"])
fds = [(set(["name"]), "color"),
       (set(["category"]), "department"),
       (set(["color", "category"]), "price")]

print_setup(attributes, fds)

Attributes = {color,price,category,department,name}
FDs = 	name -> color
	category -> department
	category,color -> price


### Closure of a set of Attributes

Let's implement the algorithm for obtaining the closure of a set of attributes:

In [10]:
def compute_closure(x, fds, verbose=False):
    # We will repeat until there are no changes.
    bChanged = True
    
    # Make a copy of the input to hold x^{+}
    x_ret = x.copy()
    
    while bChanged:
        # Must change on each iteration
        bChanged = False
        # loop through all the FDs.
        for fd in fds:
            # recall: lhs -> rhs
            (lhs, rhs) = map(to_set, fd)
            if fd_applies_to(fd, x_ret) and not rhs.issubset(x_ret):
                x_ret = x_ret.union(rhs)
                if verbose:
                    print("Using FD " + fd_to_str(fd))
                    print("\t Updated x to " + set_to_str(x_ret))
                bChanged = True

    return x_ret

As an example, let's compute the closure for the attribute "name":

In [12]:
A = set(["name"])
compute_closure(A, fds, verbose=True)

Using FD name -> color
	 Updated x to {name,color}


{'color', 'name'}

#### Exercise 3

Is the attribute "name" a superkey for this relation? Why?

A superkey is a set of attributes within a table whose values can be used to uniquely identify a tuple. 

To check if attribute *name* is a superkey of the relation *attributes = { name, category, color, department, price}*, we compute the closure of attribute $name$. We obtain:

\begin{equation*}
\{name\}^+ = \{name, color\}
\end{equation*}

It shows that attribute $name$ does not determine all the attributes of the relation *attributes*, then *name* is not a superkey.

### Keys and Superkeys

Next, we'll add some new functions now for finding superkeys and keys.  Recall:
* A _superkey_ for a relation $R(B_1,\dots,B_m)$ is a set of attributes $\{A_1,\dots,A_n\}$ s.t.
$$ \{A_1,\dots,A_n\} \rightarrow B_{j} \text{ for all } j=1,\dots m$$
* A _key_ is a minimal (setwise) _superkey_

The algorithm to determine if a set of attributes $A$ is a superkey for $X$ is actually very simple (check if $A^+=X$):

In [21]:
def is_superkey_for(A, X, fds, verbose=False):
    return X.issubset(compute_closure(A, fds, verbose=verbose))

Is "name" a superkey of the relation?

In [22]:
A = set([ "price", "name"])
is_superkey_for(A, attributes, fds)

False

Then, to check if $A$ is a key for $X$, we just confirm that:
* (a) it is a superkey
* (b) there are no smaller superkeys (_Note that we only need to check for superkeys of one size smaller_)

In [23]:
import itertools


def is_key_for(A, X, fds, verbose=False):
    subsets = set(itertools.combinations(A, len(A)-1))
    return is_superkey_for(A, X, fds) and \
        all([not is_superkey_for(set(SA), X, fds) for SA in subsets])

Now, let's look at another example:

attributes = { cru, type, client, remise}

and functional dependencies:

cru $\rightarrow$ type

type, client $\rightarrow$ remise

#### Exercise 4

Is "cru" and "client" a key of the relation? Why?

In [35]:
# These are the attribute set.
attributes = set(["cru", "type", "client", "remise"])
fds = [(set(["cru"]), "type"),
       (set(["type", "client"]), "remise")]

#print_setup(attributes, fds)

A = set([ "cru", "client"])

# Is a superkey ?
print("Is A = {} a SUPERKEY for X = {} ? {}".format(A, attributes, is_superkey_for(A, attributes, fds)))

# Is a key ?
print("Is A = {} a KEY for X = {} ? {}".format(A, attributes, is_key_for(A, attributes, fds, verbose=True)))

Is A = {'client', 'cru'} a SUPERKEY for X = {'remise', 'client', 'cru', 'type'} ? True
Is A = {'client', 'cru'} a KEY for X = {'remise', 'client', 'cru', 'type'} ? True


*{client, cru}* is a key of the relation; it is a superkey AND it is minimal (no smaller). 

### Closure of a set of functional dependencies

The algorithm to find the closure of a set of functional dependencies is the following:

In [36]:
import itertools


def findsubsets(S, m):
    return set(itertools.combinations(S, m))


def closure(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size)
        for SA in subsets:      # loop through all the subsets.
            cl = compute_closure(set(SA), fds, verbose)
            if len(cl.difference(SA)) > 0:
                c.extend([(set(SA), cl.difference(SA))])
    return c

Let's see some examples of how to use it:

In [37]:
attributes1 = set(['A', 'B', 'C', 'D'])
fds1 = [(set(['A', 'B']), 'C'),
        (set(['A', 'D']), 'B'),
        (set(['B']), 'D')]

print_fds(closure(attributes1, fds1))

FDs = 	B -> D
	C,B -> D
	A,B -> C,D
	A,D -> C,B
	C,A,B -> D
	A,B,D -> C
	C,A,D -> B


In [38]:
attributes2 = set(['CRU', 'TYPE', 'CLIENT', 'REMISE'])
fds2 = [(set(['CRU']), 'TYPE'),
        (set(['TYPE', 'CLIENT']), 'REMISE')]

print_fds(closure(attributes2, fds2))

FDs = 	CRU -> TYPE
	CLIENT,TYPE -> REMISE
	CRU,REMISE -> TYPE
	CRU,CLIENT -> REMISE,TYPE
	CRU,CLIENT,REMISE -> TYPE
	CRU,CLIENT,TYPE -> REMISE


In [39]:
attributes3 = set(['N VEH', 'TYPE', 'COULEUR', 'MARQUE', 'PUISSANCE'])
fds3 = [(set(['N VEH']), 'TYPE'),
        (set(['N VEH']), 'COULEUR'),
        (set(['TYPE']), 'MARQUE'),
        (set(['TYPE']), 'PUISSANCE')]

print_fds(closure(attributes3, fds3))

FDs = 	TYPE -> MARQUE,PUISSANCE
	N VEH -> PUISSANCE,COULEUR,TYPE,MARQUE
	PUISSANCE,TYPE -> MARQUE
	COULEUR,N VEH -> PUISSANCE,TYPE,MARQUE
	N VEH,TYPE -> PUISSANCE,COULEUR,MARQUE
	MARQUE,TYPE -> PUISSANCE
	COULEUR,TYPE -> PUISSANCE,MARQUE
	MARQUE,N VEH -> PUISSANCE,COULEUR,TYPE
	PUISSANCE,N VEH -> COULEUR,TYPE,MARQUE
	PUISSANCE,N VEH,TYPE -> COULEUR,MARQUE
	MARQUE,COULEUR,TYPE -> PUISSANCE
	PUISSANCE,MARQUE,N VEH -> COULEUR,TYPE
	COULEUR,N VEH,TYPE -> PUISSANCE,MARQUE
	MARQUE,N VEH,TYPE -> PUISSANCE,COULEUR
	MARQUE,COULEUR,N VEH -> PUISSANCE,TYPE
	PUISSANCE,COULEUR,TYPE -> MARQUE
	PUISSANCE,COULEUR,N VEH -> TYPE,MARQUE
	PUISSANCE,COULEUR,N VEH,TYPE -> MARQUE
	PUISSANCE,MARQUE,N VEH,TYPE -> COULEUR
	MARQUE,COULEUR,N VEH,TYPE -> PUISSANCE
	PUISSANCE,COULEUR,MARQUE,N VEH -> TYPE


Now, let's write a method to find superkeys of the relations:


In [40]:
def superkeys(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size)
        for SA in subsets:
            cl = compute_closure(set(SA), fds, verbose)
            if cl == X and len(cl.difference(SA)) > 0:  # cl = X
                c.extend([SA])

    return c

In [41]:
superkeys(attributes1, fds1)

[('A', 'B'), ('A', 'D'), ('C', 'A', 'B'), ('A', 'B', 'D'), ('C', 'A', 'D')]

Let's see some examples:

In [42]:
superkeys(attributes2, fds2)

[('CRU', 'CLIENT'), ('CRU', 'CLIENT', 'REMISE'), ('CRU', 'CLIENT', 'TYPE')]

In [43]:
superkeys(attributes3, fds3)

[('N VEH',),
 ('COULEUR', 'N VEH'),
 ('N VEH', 'TYPE'),
 ('MARQUE', 'N VEH'),
 ('PUISSANCE', 'N VEH'),
 ('PUISSANCE', 'N VEH', 'TYPE'),
 ('PUISSANCE', 'MARQUE', 'N VEH'),
 ('COULEUR', 'N VEH', 'TYPE'),
 ('MARQUE', 'N VEH', 'TYPE'),
 ('COULEUR', 'MARQUE', 'N VEH'),
 ('PUISSANCE', 'COULEUR', 'N VEH'),
 ('PUISSANCE', 'COULEUR', 'N VEH', 'TYPE'),
 ('PUISSANCE', 'MARQUE', 'N VEH', 'TYPE'),
 ('COULEUR', 'MARQUE', 'N VEH', 'TYPE'),
 ('PUISSANCE', 'COULEUR', 'MARQUE', 'N VEH')]

#### Exercise 5

Implement a `keys` method to find keys of a relation.

**Note**: If there exist multiple keys of a relation, the `keys` method should return at least one of them.

In [50]:
def keys(attributes, fds, verbose=False):
    
    key = None
    
    # The method superkeys() outputs superkeys in ascending order,
    # ie smallest key is first
    super_keys = superkeys(attributes, fds, verbose=verbose)
    
    # Check if c is not empty
    if len(super_keys) > 0:
        key = super_keys[0]
    
    return key

Test it 

In [54]:
keys(attributes1, fds1, verbose=False)

Using FD B -> D
	 Updated x to {B,D}
Using FD B -> D
	 Updated x to {C,B,D}
Using FD A,B -> C
	 Updated x to {C,A,B}
Using FD B -> D
	 Updated x to {C,A,B,D}
Using FD A,D -> B
	 Updated x to {A,B,D}
Using FD A,B -> C
	 Updated x to {C,A,B,D}
Using FD B -> D
	 Updated x to {C,A,B,D}
Using FD A,B -> C
	 Updated x to {C,A,B,D}
Using FD A,D -> B
	 Updated x to {C,A,B,D}


('A', 'B')

In [53]:
keys(attributes2, fds2)

('CRU', 'CLIENT')

In [52]:
keys(attributes3, fds3)

('N VEH',)